## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Получение первых 5 строк таблицы для наглядного представления 

In [2]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Каждая строка таблицы содержит подробную информацию о клиентах банка, включая семейное положение, количество детей и ежемесячный доход. Первоначальная проблема, которую необходимо решить пропуски и удаление дубликатов. Далее для проверки рабочих гипотез особенно ценны столбцы children, family_status_id, total_income и purpose.

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [3]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пустые значения свидетельствуют, что по некоторым клиентам доступна не вся информация по трудовому стажу и ежемесячному доходу. Причины могут быть разные: человеческий фактор или проблемы с записью данных. Нас интересует обработка пропусков только по столбцу с доходом, т.к одна из стоящих задач заключается в определении зависимости между уровнем дохода и возвратом кредита в срок.

Предположим, что сотрудники одного и того же типа занятости имеют примерно одинаковый доход. И заполним пропуски медианой по этим группам.

In [4]:
# <Цикл по замене пустых значений в столбце доходов на медианы по типу занятости>
types = df['income_type'].unique()

for i in types:
    median = df[df['income_type'] == i]['total_income'].median()
    df.loc[df['income_type'] == i, 'total_income'] = df.loc[df['income_type'] == i, 'total_income'].fillna(median)

Убеждаемся, что замена произошла

In [5]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Вывод

Были обноружены пустые значения в столбце с ежемесячным доходом. Поскольку эти сведения важны для дальнейшего анализа, не просто удаляем все пропущенные значения, а заполняем медианой ежемесячного дохода по типу занятости

### Замена типа данных

Всего в таблице 12 столбцов. Столбцы days_employed и total_income рекомендуется заменить с float64 на int64, т.к. int64 затрачивает меньше памяти при вычислениях. Заменим данные только в столбце total_income, т.к. столбец days_employed нам не интересен для анализа.

In [6]:
# <Замена типа данных>
df['total_income'] = df['total_income'].astype('int')

In [7]:
# <Проверка замены>
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод
Заменили в одном столбце тип данных для освобождения памяти. Тип данных у всех столбцов указан верно. 

### Обработка дубликатов

Необходимо установить наличие дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.

In [8]:
# <получение суммарного количества дубликатов в таблице df>
df.duplicated().sum()

54

In [9]:
# <получение суммарного количества дубликатов в таблице df>
df[df.duplicated(keep=False)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,NaN,46,среднее,1,женат / замужем,0,F,сотрудник,0,142594,высшее образование
520,0,NaN,35,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
541,0,NaN,57,среднее,1,женат / замужем,0,F,сотрудник,0,142594,сделка с подержанным автомобилем
554,0,NaN,60,среднее,1,женат / замужем,0,M,сотрудник,0,142594,покупка недвижимости
680,1,NaN,30,высшее,0,женат / замужем,0,F,госслужащий,0,150447,покупка жилья для семьи
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,NaN,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием
21132,0,NaN,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью
21281,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости


In [10]:
# <удаление всех дубликатов из таблицы df специальным методом>
df = df.drop_duplicates().reset_index(drop=True)

In [11]:
# <проверка на отсутствие>
df.duplicated().sum()

0

Проверим данные в столбце children на корректность

In [12]:
# <Проверка на отрицательные значения>
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Имеются некорректные данные (-1 и 20). Предположим, что это ошибка в занесении данных из-за человеческий фактора и произведем замену наиболее вероятных значений по смыслу

In [13]:
# <Замена некорректных значений в столбце children>
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 1)

In [14]:
# <Проверка замены>
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Вывод

На этапе предобработки в данных обнаружились не только пропуски, но и дубликаты и некорректные значения. Дубликаты могли появиться вследствие сбоя в записи данных. Их удаление позволит провести анализ точнее. Некорректные значения могли появится вследствии человеческого фактора. Предобработка завершена, приступим к дальнейшей работе. 

### Лемматизация

In [15]:
# <Импорт библиотеки для лемматизации>
from pymystem3 import Mystem
m = Mystem()

In [16]:
# <Cохраняем уникальные значения целей кредита в отдельную переменную>
purpose_word = df['purpose'].unique() 

In [17]:
# <Объединим значения целей в единую строку>
purpose_string = ' '.join(purpose_word)

In [18]:
# <Применим лемматизацию>
lemmas = m.lemmatize(purpose_string)

In [19]:
# <Выведем количество часто встречающихся слов>
from collections import Counter
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


In [20]:
# <Выведем количество уникальных значений по столбцу purpose>
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [21]:
# <Составления массива из основных значений>
purpose_category = ['автомобиль', 'образование', 'свадьба', 'недвижимость']

In [22]:
def purpose_group(purpose_category):
        purpose_list = m.lemmatize(purpose_category) #лемматизируем исходную строку
        for name in purpose_list: #если среди строки есть с ключевым словом, тогда заменить строку на ключевое слово
            if name == 'жилье':
                purpose_string = 'недвижимость'
                return purpose_string
            if name == 'автомобиль':
                purpose_string = 'автомобиль'
                return purpose_string
            if name == 'недвижимость':
                purpose_string = 'недвижимость'
                return purpose_string
            if name == 'образование':
                purpose_string = 'образование'
                return purpose_string
            if name == 'свадьба':
                purpose_string = 'свадьба'
                return purpose_string

df['purpose_group'] = df['purpose'].apply(purpose_group)

In [23]:
# <Проверка на отсутствие пустых значений в столбце purpose_group>
df['purpose_group'].isnull().sum()

0

### Вывод

Провели лемматизацию, выделив 4 категории по цели получения кредита. Добавили в исходный файл новый столбец с указанными категориями для дальнейшего анализа.

### Категоризация данных

Распределим заемщиков кредитов на 3 категории по числу детей:

In [24]:
def children_group(children):
        """
        Возвращает возврастную группу по значению возраста age, используя правила:
        - 'детей нет' при значении age < 1 лет
        - 'один ребенок' при значении age < 2 лет
        - 'многодетная семья' во всех остальных случаях
        """
        if children < 1:
                return 'детей нет'
        if children < 2:
                return 'один ребенок'
        return 'многодетная семья'

In [25]:
# <Добавление столбца children_group в исходную таблицу и проверка работы функции>
df['children_group'] = df['children'].apply(children_group)

### Вывод

В исходную таблицу добавили 2 новых столбца по категориям количества детей и целям получения кредита

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
# <Создание сводной таблицы по количеству детей и возврату кредита. Доля считается от всех остальных по данной категории> 
children_pivot = df.pivot_table(index='children_group', columns='debt', values='family_status_id', aggfunc='count')
children_pivot['ratio'] = children_pivot[1] / (children_pivot[0] + children_pivot[1]) 
children_pivot['ratio'] = children_pivot['ratio'].apply(lambda x: '{:.2%}'.format(x))
children_pivot.sort_values(by = 'ratio', ascending=False)

debt,0,1,ratio
children_group,,,
многодетная семья,2207,225,9.25%
один ребенок,4479,453,9.18%
детей нет,13044,1063,7.54%


### Вывод
У многодетных семей процент возврата кредита ниже, чем у семей без детей.  Разница составляет 1,78%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
# <Создание сводной таблицы по семейному положению и возврату кредита. Доля считается от всех остальных по данной категории> 
family_status_pivot = df.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
family_status_pivot['ratio'] = family_status_pivot[1] / (family_status_pivot[0] + family_status_pivot[1]) 
family_status_pivot['ratio'] = family_status_pivot['ratio'].apply(lambda x: '{:.2%}'.format(x))
family_status_pivot.sort_values(by = "ratio", ascending=False)

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,9.75%
гражданский брак,3775,388,9.32%
женат / замужем,11413,931,7.54%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%


Лучше всего возвращают кредиты люди находящиеся в разводе и потерявшие супруга/супругу.

### Вывод
Самый большой процент

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
# <Использование квантили нормального распределения для категоризации данных по уровню дохода>
pd.options.display.float_format = '{:.0f}'.format
df['total_income'].describe()

count     21471
mean     165295
std       98153
min       20667
25%      107654
50%      142594
75%      195768
max     2265604
Name: total_income, dtype: float64

In [29]:
def income_group(income):
        """
        Возвращает возврастную группу по значению возраста age, используя правила:
        - 'детей нет' при значении age < 1 лет
        - 'дети есть' во всех остальных случаях
        """
        if income < 103053:
                return 'низкий'
        if income <= 195768:
                return 'средний'
        return 'высокий'

In [30]:
# <Добавление столбца children_group в исходную таблицу и проверка работы функции>
df['income_group'] = df['total_income'].apply(income_group)

In [31]:
total_income = df.pivot_table(index='income_group', values='debt')
total_income['debt'] = total_income['debt'].apply(lambda x: '{:.2%}'.format(x))
total_income.sort_values(by = "debt", ascending=False)

,debt
income_group,
средний,8.66%
низкий,7.92%
высокий,7.13%


У людей со среднем ежемесячным доходом самый большой процент не возврата. Разница с людьми с высоким дохожом составляет 1,35%.

### Общий вывод

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
purpose_group_pivot = df.pivot_table(index='purpose_group', values='debt')
purpose_group_pivot['debt'] = purpose_group_pivot['debt'].apply(lambda x: '{:.2%}'.format(x))
purpose_group_pivot.sort_values(by = "debt", ascending=False)

,debt
purpose_group,
автомобиль,9.35%
образование,9.22%
свадьба,7.97%
недвижимость,7.23%


У людей берущих кредит на машину, самый большой процент не возврата. Разница с недвижимостью (лучший процент возврата) составляет  2,12%.

### Шаг 4. Общий вывод

В результате анализа выявлена зависимость между задолжностью по возврату кредита и наличием детей, семеным положением и уровнем дохода. Так же определено, что у людей берущих кредит на машину, самый большой процент не возврата кредита в срок. Составим самый не благополучный портрет кредитора - не женатый человек, со средний доходом и берущий кредит на автомобиль.